<a href="https://colab.research.google.com/github/kaz-yamam0t0/wav2lip-test/blob/main/wav2lip_sample02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
!pip3 install --quiet ml4a
!pip3 install --quiet moviepy

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1mlB3Qyuchyq74Nf3jcjCqn9J6U61zWWc', './wav2lip-samples.zip', quiet=False)
! unzip wav2lip-samples.zip

In [ ]:
from ml4a import audio
from ml4a import image
from ml4a.models import wav2lip
import glob, os

if not os.path.isdir("./tmp"): 
	os.makedirs("./tmp")

if not os.path.isdir("./dist"): 
	os.makedirs("./dist")

audio = "./wav2lip-samples/koike.mp3"
images = glob.glob("./wav2lip-samples/*.jpg")[:(1+4+16+25)*3]
index = 0
for image in images:
	index += 1
	dst = ('./tmp/%04d.mp4' % index)
	if not os.path.isfile(dst):
		wav2lip.run(
			image, 
			audio,
			output_video = dst, 
			pads = [0, 0, 0, 0],
			resize_factor = 1, 
			crop = None, 
			box = None, 
		)

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, clips_array

videos = []
index = 1

def load(n=1, t_start=None, t_end=None):
	global videos, index
	rows = []
	for x in range(n):
		cols = []
		for y in range(n):
			v_ = VideoFileClip("./tmp/%04d.mp4" % index).subclip(t_start=t_start,t_end=t_end)
			index += 1

			if x != 0 or y != 0: v_ = v_.volumex(0)
			cols.append(v_)
		rows.append(cols)
	
	videos.append(clips_array(rows).resize(width=1280))

for i in range(3):
	load(1,t_start=0.5,t_end=1.3)
	load(2,t_start=1.3,t_end=1.9)
	load(3,t_start=1.9,t_end=2.5)
	load(4,t_start=2.5,t_end=3.0)

v0 = concatenate_videoclips(videos).resize(width=1280)
v0.write_videofile('./dist/concatenated.mp4')

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('dist/concatenated.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="40%" height="40%" controls>
	<source src="{data_url}" type="video/mp4">
</video>""")